# Exercises 3, answers

In [ ]:
def objective_function(x): # function to be optimized
    return (x[0]**2.0 + x[1]**2.0 + x[0] + 2.0 * x[1])

$\nabla f(x) = (2x_1+1,2x_2+2)=0$ if and only if $x_1=-0.5$ and $x_2 = -1.0$. Thus $x^*=(-0.5,-1.0)$.

## Task 1
* max 3 points, 3 points if one gets correct solution and line search is properly done, reductions if something is not done correctly

In [ ]:
import math
def golden_section_line_search(a,b,f,L): # same as in the lectures
    x = a
    y = b
    while y-x>2*L:
        if f(x+(math.sqrt(5.0)-1)/2.0*(y-x))<f(y-(math.sqrt(5.0)-1)/2.0*(y-x)):
            x = y-(math.sqrt(5.0)-1)/2.0*(y-x)
        else:
            y = x+(math.sqrt(5.0)-1)/2.0*(y-x)
    return (x+y)/2

In [ ]:
import numpy as np
import ad
def steepest_descent_withgolden(f,start,search_interval_length,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    d = float('Inf')
    while np.linalg.norm(d)>precision:
        f_old = f_new
        d = -np.array(ad.gh(f)[0](x)) # search direction of the steepest descent
        # step length optimization with golden section
        step = golden_section_line_search(0,
                                          search_interval_length/np.linalg.norm(d), # normalization, can be done earlier as well
                                          lambda t: f(x+t*d), # function with respect to step length t
                                          precision)
        x = x+d*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [ ]:
start = [-5,10]
precision = 0.0001
(x,f_new,steps1) = steepest_descent_withgolden(objective_function,
                                               start,10,precision)
print (x)
print(len(steps1))
print(steps1)

## Task 2
* max 2 points, 1 point for plot, 1 point if there is comparison/analysis

In [ ]:
import numpy as np
import ad
def steepest_descent(f,start,step,precision): # from the lectures
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    d = float('Inf')
#    while abs(f_old-f_new)>precision and len(steps)<10:
    while np.linalg.norm(d)>precision and len(steps)<20:
        f_old = f_new
        d = -np.array(ad.gh(f)[0](x))
        x = x+d*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [ ]:
step = 0.2
(x,f_new,steps2) = steepest_descent(objective_function,start,step,precision)
print(x)
print(len(steps2))
print(steps2)

In [ ]:
import matplotlib.pyplot as plt

def plot_2d_steps2(steps1,start1,steps2,start2):
    myvec1 = np.array([start1]+steps1).transpose()
    myvec2 = np.array([start2]+steps2).transpose()
    plt.plot(myvec2[0,],myvec2[1,],'bo')    
    plt.plot(myvec1[0,],myvec1[1,],'rx')
    return plt

In [ ]:
plot_2d_steps2(steps1,start,steps2,start).show() # optimized blue, fixed red

**Remarks**
* With fixed step size the steps are not actually of equal length since the step size is multiplied with the gradient whose length varies
* The performance of steepest descent with optimized step size also depends on what is set as the maximum step length
  * too long --> golden section search uses a high nuber of function evaluations
  * too short --> takes shorter steps where improvement could still be made with longer ones

## Task 3
* max 3 points, 3 points if one gets correct result, reductions is there are flaws

$$y_k=\nabla f(x_k+s_k)-\nabla f(x_k),$$

$$s_k=x_{k+1} -x_k=(x_k+s_k)-x_k$$

$$H_{k+1}=H_{k}-\frac{H_k y_k y_k^T H_k}{y_k^T H_k y_k}+\frac{s_k s_k^T}{y_k^{T} s_k}$$

In [ ]:
import ad
import numpy as np
def update_Hinv(H_inv_old,x_old,x_new,f): # subroutine to update inverse of the Hessian
    y = np.matrix(ad.gh(f)[0](x_new)-ad.gh(f)[0](x_old)).transpose() # compute y_k
    second_term = H_inv_old*y*y.transpose()*H_inv_old.transpose()/(y.transpose()*H_inv_old*y)
    s = np.matrix(x_new-x_old).transpose() # compute s_k
    third_term = s*s.transpose()/(y.transpose()*s)
    H_inv_new = H_inv_old-second_term+third_term
    return H_inv_new

In [ ]:
import numpy as np
def quasi_newton_DFP(f,start,step,precision):
    f_old = float('Inf')
    x_new = np.array(start)
    steps = []
    f_new = f(x_new)
    # Use identity matrix as the first approximation (is positive definite)
    H_inv = np.eye(len(start))
    d = float('Inf')
#    while abs(f_old-f_new)>precision and len(steps)<10:
    while np.linalg.norm(d)>precision and len(steps)<20:
        d = (-H_inv*(np.matrix(ad.gh(f)[0](x_new)).transpose())).transpose()
        x_old = x_new
        f_old = f_new
        #Change the type from np.matrix to np.array so that we can use it in our function
        x_new = np.array(x_new+d*step)[0]
        f_new = f(x_new)
        steps.append(list(x_new))
        H_inv = update_Hinv(H_inv,x_old,x_new,f)
    return x_new,f_new,steps

In [ ]:
start = [-3,2]
(x,f_new,steps)=quasi_newton_DFP(objective_function,start,0.5,precision)
print(x)
print(len(steps))
print (steps)

In [ ]:
def plot_2d_steps(steps,start):
    myvec = np.array([start]+steps).transpose()
    plt.plot(myvec[0,],myvec[1,],'bo')
    return plt

In [ ]:
plot_2d_steps(steps,start).show()